In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
#from nltk.stem.snowball import SnowballStemmer #0.003 improvement but takes twice as long as PorterStemmer
#stemmer = SnowballStemmer('english')
import re
#import enchant
import random
random.seed(2016)
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#import enchant



In [120]:
def isFloat(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def eval_f(expr):
    first = ''
    second = ""
    flag = 0

    if expr.find("/") == 1:    
        for a in expr:            
            if a == "/":
                flag = 1
                continue
            if flag == 0:
                first += a
            if flag == 1:
                second += a
        
        if (first.isnumeric() == True and  second.isnumeric() == True):
            if (float(second)!= 0):
                return (float(first)/float(second))
            else:
                return 1000
    else:
        if (expr.isnumeric()):
            return float(expr)
        else:
            return 1

In [134]:
def multiply_search(exp):
    if isFloat(exp):
        return 0
    a = exp.split()
    
    prod = 0
    for i in range(0,len(a)):
        if a[i] == 'xbi':        
            
            if (i+1 != len(a)):
                #print (a[i-1],a[i+1])
                try:
                    prod = eval_f(re.sub("[a-z.]", "",a[i-1])) * eval_f(re.sub("[a-z.]", "",a[i+1]))
                except Exception as e:
                    pass
                
                a[i] = str(prod)
                #print(prod)
   
    return (" ".join(a))   

def to_float(exp):
    if isinstance(exp, str): 
        s = (" ").join([str(float(z)) for z in exp.split(" ") if isFloat(z) ])        
        return s
def to_str(exp):
    if isinstance(exp, str): 
        s = (" ").join([z for z in exp.split(" ") if not isFloat(z) ])
        return s

    
  


In [12]:
def prod_attr_score(uid, search_term):
    if isinstance(search_term, str): 
        attr_list  = list()
        count = 0
        total_len = 0
        for a in df_attr[df_attr.product_uid == uid].iterrows():            
                if not a[1][1] is None and a[1][1].find("Bullet") == 0 :
                    if not a[1][2] is None:
                        for b in  (a[1][2].split()):
                            if isFloat(b) is False:
                                attr_list.append(b)
        #print (attr_list, search_term)      
        
        for word in search_term.split():
            if isFloat(word) is True:
                continue
            if word in attr_list:
                count += 1
            total_len += 1
        #print(count)        
        return count
        


In [13]:
def prod_attr_score_num(uid, search_term):
    unit_list = ["(in.)", "(lb.)", "(volts.)", "(watts)","sq." ,"(amps)","(min.)","(hours)","(ft.)" ]
    if isinstance(search_term, str): 
        flag = 0
        attr_list  = list()
        count = 0
        total_len = 0
        for a in df_attr[df_attr.product_uid == uid].iterrows():            
                if not a is None:
                    for name_word in a[1][1].split():
                        if name_word in unit_list or (a[1][2] is not None and isFloat(a[1][2]) == True):
                            flag = 1
                        else:
                            flag = 0
                    if not a[1][2] is None and flag == 1:
                        for b in  (a[1][2].split()):
                            if isFloat(b) is True:
                                attr_list.append(float(b))
        #print (attr_list, search_term)      
        #print (attr_list,search_term)
        for word in search_term.split():
            if isFloat(word) is False:
                continue
            if float(word) in attr_list:
                count += 1
            total_len += 1
        #print(count)        
            return count
        


In [129]:
df_train = pd.read_excel('pre_processed_data\df_train.xlsx')
df_test = pd.read_excel('pre_processed_data\df_test.xlsx')
df_prod = pd.read_excel('pre_processed_data\df_prod.xlsx')
df_brand = pd.read_excel('pre_processed_data\df_brand.xlsx')
df_attr = pd.read_excel('pre_processed_data\df_attr.xlsx')

In [ ]:
df_train['attr_bullet_score']  = df_train.apply(lambda df_train: prod_attr_score(df_train['product_uid'],df_train['search_term'] ) ,axis = 1)

df_test['attr_bullet_score']  = df_test.apply(lambda df_test: prod_attr_score(df_test['product_uid'],df_test['search_term'] ) ,axis = 1)

df_train['attr_bullet_score_num']  = df_train.apply(lambda df_train: prod_attr_score_num(df_train['product_uid'],df_train['search_term'] ) ,axis = 1)

df_test['attr_bullet_score_num']  = df_test.apply(lambda df_test: prod_attr_score_num(df_test['product_uid'],df_test['search_term'] ) ,axis = 1)

In [6]:
train_X = df_train[['prod_title_score','prod_brand_score']]#,'attr_bullet_score','attr_bullet_score_num']]
train_Y = df_train[['relevance']]

test_X = df_test[['prod_title_score','prod_brand_score']]#,'attr_bullet_score','attr_bullet_score_num']]

train_X = train_X.fillna(0)
test_X = test_X.fillna(0)

In [7]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(random_state=2016, n_estimators=1000)
clf.fit(train_X, np.array(train_Y['relevance'].values.tolist()))
out = (clf.predict(test_X))
df_out = pd.DataFrame(columns=['id','relevance'])
df2 = pd.DataFrame({'id': [1], 'relevance': 2})
df_out = df_out.append(df2)

df_r1 = df_test[['id']]
df_r2 = pd.DataFrame(out, columns=['relevance'])
result = pd.concat([df_r1, df_r2], axis=1)
result.to_csv("results5.csv")


In [144]:
df_test['search_term'] = df_test['search_term'].apply(multiply_search)
df_test['search_term_num'] = df_test['search_term'].apply(to_float)
df_test['search_term_str'] = df_test['search_term'].apply(to_str)


df_train['search_term'] = df_train['search_term'].apply(multiply_search)
df_train['search_term_num'] = df_train['search_term'].apply(to_float)
df_train['search_term_str'] = df_train['search_term'].apply(to_str)



In [131]:
df_attr['value'] = df_attr['value'].apply(multiply_search)
df_attr['value_num'] = df_attr['value'].apply(to_float)
df_attr['value_str'] = df_attr['value'].apply(to_str)

In [132]:
df_attr.head()

,product_uid,name,value,value_num,value_str
0,100001,Bullet01,versatil connector variou 90 degre connect hom...,90.0,versatil connector variou degre connect home r...
1,100001,Bullet02,stronger angl nail screw fasten alon,,stronger angl nail screw fasten alon
2,100001,Bullet03,help ensur joint consist straight strong,,help ensur joint consist straight strong
3,100001,Bullet04,dimensions: 3 in. x 3 in. x 1 1/2 in.,3.0 3.0 1.0,dimensions: in. x in. x 1/2 in.
4,100001,Bullet05,made 12 gaug steel,12.0,made gaug steel


In [130]:
df_prod.head()

,product_uid,product_description
0,100001,angl make joint stronger also provid consist s...
1,100002,behr premium textur deckov innov solid color c...
2,100003,classic architectur meet contemporari design e...
3,100004,grape solar 265watt. polycrystallin pv solar p...
4,100005,updat bathroom delta vero singl handl shower f...


In [135]:
df_prod['product_description'] = df_prod['product_description'].apply(multiply_search)
df_prod['product_description_num'] = df_prod['product_description'].apply(to_float)
df_prod['product_description_str'] = df_prod['product_description'].apply(to_str)

In [123]:
eval_f("1/0")

1000

In [136]:
df_prod.head()

,product_uid,product_description,product_description_num,product_description_str
0,100001,angl make joint stronger also provid consist s...,90.0 9.0 3.0 1.0 12.0 10.0 1.0 1.0,angl make joint stronger also provid consist s...
1,100002,behr premium textur deckov innov solid color c...,2.0,behr premium textur deckov innov solid color c...
2,100003,classic architectur meet contemporari design e...,124.1 2.0 45.1,classic architectur meet contemporari design e...
3,100004,grape solar 265watt. polycrystallin pv solar p...,4.0 4.0,grape solar 265watt. polycrystallin pv solar p...
4,100005,updat bathroom delta vero singl handl shower f...,3.0 10000.0,updat bathroom delta vero singl handl shower f...


In [138]:
df_test.head()

,id,product_uid,product_title,search_term,prod_title_score,prod_brand_score,attr_bullet_score,attr_bullet_score_num
0,1,100001,simpson strong tie 12 gaug angl,90deg. bracket,0.000000,0.000000,0.000000,0
1,4,100001,simpson strong tie 12 gaug angl,metal l bracket,0.000000,0.000000,0.000000,0
2,5,100001,simpson strong tie 12 gaug angl,simpson sky abl,0.333333,0.333333,0.000000,0
3,6,100001,simpson strong tie 12 gaug angl,simpson strong tie,1.000000,0.333333,0.333333,0
4,7,100001,simpson strong tie 12 gaug angl,simpson strong tie hcc 668,0.600000,0.200000,0.250000,0


In [141]:
df_test = df_test.drop(['prod_title_score','prod_brand_score','attr_bullet_score','attr_bullet_score_num'],1)

In [142]:
df_train = df_train.drop(['prod_title_score','prod_brand_score','attr_bullet_score','attr_bullet_score_num'],1)

In [149]:
df_test.head()

,id,product_uid,product_title,search_term,search_term_num,search_term_str
0,1,100001,simpson strong tie 12 gaug angl,90deg. bracket,,90deg. bracket
1,4,100001,simpson strong tie 12 gaug angl,metal l bracket,,metal l bracket
2,5,100001,simpson strong tie 12 gaug angl,simpson sky abl,,simpson sky abl
3,6,100001,simpson strong tie 12 gaug angl,simpson strong tie,,simpson strong tie
4,7,100001,simpson strong tie 12 gaug angl,simpson strong tie hcc 668,668.0,simpson strong tie hcc


In [150]:
df_test['product_title'] = df_test['product_title'].apply(multiply_search)
df_test['product_title_num'] = df_test['product_title'].apply(to_float)
df_test['product_title_str'] = df_test['product_title'].apply(to_str)


df_train['product_title'] = df_train['product_title'].apply(multiply_search)
df_train['product_title_num'] = df_train['product_title'].apply(to_float)
df_train['product_title_str'] = df_train['product_title'].apply(to_str)



In [155]:
df_attr.head()

,product_uid,name,value,value_num,value_str
0,100001,Bullet01,versatil connector variou 90 degre connect hom...,90.0,versatil connector variou degre connect home r...
1,100001,Bullet02,stronger angl nail screw fasten alon,,stronger angl nail screw fasten alon
2,100001,Bullet03,help ensur joint consist straight strong,,help ensur joint consist straight strong
3,100001,Bullet04,dimensions: 3 in. x 3 in. x 1 1/2 in.,3.0 3.0 1.0,dimensions: in. x in. x 1/2 in.
4,100001,Bullet05,made 12 gaug steel,12.0,made gaug steel


In [157]:
df_train.to_excel('pre_processed_data\df_train.xlsx')
df_test.to_excel('pre_processed_data\df_test.xlsx')
df_prod.to_excel('pre_processed_data\df_prod.xlsx')
df_attr.to_excel('pre_processed_data\df_attr.xlsx')
df_brand.to_excel('pre_processed_data\df_brand.xlsx')